In [24]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random

# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model
# from keras.layers import (
#     Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,
#     GlobalAveragePooling2D
# )
# from keras.optimizers import Adam
# from keras.applications import ResNet50
# from keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,
    GlobalAveragePooling2D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical





In [25]:
def load_labels(base_dir, num_samples=10):
    txt_data = {}
    label_files = []
    
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".txt"):
                label_files.append(os.path.join(root, file))
    
    selected_label_files = random.sample(label_files, min(num_samples, len(label_files)))
    
    for file_path in selected_label_files:
        try:
            with open(file_path, 'r') as f:
                txt_data[file_path] = f.read()
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    print(f"Loaded {len(txt_data)} TXT files from {base_dir}")
    return txt_data

def load_images(base_dir, num_samples=10):
    images = []
    filenames = []
    image_files = []
    
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    
    selected_image_files = random.sample(image_files, min(num_samples, len(image_files)))
    
    for file_path in selected_image_files:
        try:
            image = Image.open(file_path).resize((224, 224)).convert('RGB')
            images.append(np.array(image))
            filenames.append(file_path)
        except UnidentifiedImageError as e:
            print(f"Error loading image {file_path}: {e}")
    
    print(f"Loaded {len(images)} images from {base_dir}")
    return np.array(images), filenames


In [26]:
BASE = "Database_Realtime"
IMU = os.path.join(BASE, "IMU")
VISION = os.path.join(BASE, "Vision/Webcam")

print("Loading Labels...")
txt_files = load_labels(IMU, num_samples=10)

print("Loading Images...")
images, image_paths = load_images(VISION, num_samples=10)

print("Image shape:", images.shape)
print("Labels:", len(txt_files))


Loading Labels...
Loaded 10 TXT files from Database_Realtime\IMU
Loading Images...
Loaded 10 images from Database_Realtime\Vision/Webcam
Image shape: (10, 224, 224, 3)
Labels: 10


In [27]:
def parse_labels(txt_data):
    labels = []
    for file_path, data in txt_data.items():
        lines = data.splitlines()
        for line in lines:
            values = line.split()
            if len(values) >= 4:
                pose = int(values[0]) 
                depth = float(values[1])  
                pitch = float(values[2])  
                roll = float(values[3])  
                labels.append([pose, depth, pitch, roll])
    return np.array(labels)

y_labels = parse_labels(txt_files)
print("Parsed Labels Shape:", y_labels.shape)


Parsed Labels Shape: (315, 4)


In [28]:


X_images = images / 255.0  

y_pose = y_labels[:X_images.shape[0], 0]  
depth_labels = y_labels[:X_images.shape[0], 1]  

y_labels = np.stack((y_pose, depth_labels), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)




Training set shape: (6, 224, 224, 3)
Validation set shape: (2, 224, 224, 3)
Test set shape: (2, 224, 224, 3)


In [29]:
def create_cnn_model(input_shape, num_classes=5):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_cnn_model(X_train.shape[1:], num_classes=5)
model.summary()


c:\Users\islas\OneDrive\Documents\GitHub\deep_learning_proj\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    47,776,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,797,061 (182.33 MB)

 Trainable params: 47,796,869 (182.33 MB)

 Non-trainable params: 192 (768.00 B)

In [31]:
from sklearn.preprocessing import LabelEncoder

# Combine all labels for consistent encoding
all_labels = np.concatenate((y_train[:, 0], y_val[:, 0], y_test[:, 0]), axis=0)

# Fit LabelEncoder on all labels
label_encoder = LabelEncoder()
label_encoder.fit(all_labels.astype('int'))

# Transform labels for each dataset
y_train_classes = label_encoder.transform(y_train[:, 0].astype('int'))
y_val_classes = label_encoder.transform(y_val[:, 0].astype('int'))
y_test_classes = label_encoder.transform(y_test[:, 0].astype('int'))

# Verify label mapping
print("Unique training labels:", np.unique(y_train_classes))
print("Unique validation labels:", np.unique(y_val_classes))
print("Unique test labels:", np.unique(y_test_classes))




Unique training labels: [2 3 5 6 7 9]
Unique validation labels: [0 4]
Unique test labels: [1 8]


In [32]:
def create_mlp(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

mlp_model = create_mlp((X_train.shape[1] * X_train.shape[2] * X_train.shape[3],))
history_mlp = mlp_model.fit(
    X_train.reshape(X_train.shape[0], -1), y_train[:, 1],  
    epochs=20,
    batch_size=32,
    validation_data=(X_val.reshape(X_val.shape[0], -1), y_val[:, 1]),
    verbose=1
)

test_loss_mlp, test_mae_mlp = mlp_model.evaluate(X_test.reshape(X_test.shape[0], -1), y_test[:, 1], verbose=2)
print(f"Test MAE (MLP): {test_mae_mlp}")


Epoch 1/20


c:\Users\islas\OneDrive\Documents\GitHub\deep_learning_proj\.conda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.5065 - mae: 0.5856 - val_loss: 15505.0146 - val_mae: 124.5191
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 11838.0068 - mae: 107.6721 - val_loss: 4.1412 - val_mae: 2.0284
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 81.4516 - mae: 7.6115 - val_loss: 318.9003 - val_mae: 17.8572
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 1022.5565 - mae: 26.5786 - val_loss: 306.0032 - val_mae: 17.4922
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 349.8127 - mae: 16.9535 - val_loss: 171.9475 - val_mae: 13.1128
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - loss: 152.7066 - mae: 11.1755 - val_loss: 55.8688 - val_mae: 7.4744
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 464.6259 - mae: 15.1689 - val_loss: 0.0129 - val_mae: 0.1133
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 59.6438 - mae: 7.1302 - val_loss: 36.8563 - val_mae: 6.0618
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/ste

In [33]:
def create_resnet(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(1)(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

resnet_model = create_resnet(X_train.shape[1:])
history_resnet = resnet_model.fit(
    X_train, y_train[:, 1],
    epochs=20,
    batch_size=16,
    validation_data=(X_val, y_val[:, 1]),
    verbose=1
)

test_loss_resnet, test_mae_resnet = resnet_model.evaluate(X_test, y_test[:, 1], verbose=2)
print(f"Test MAE (ResNet): {test_mae_resnet}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 84s 1us/step
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - loss: 0.1876 - mae: 0.2565 - val_loss: 0.0470 - val_mae: 0.2156
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step - loss: 1.7558 - mae: 1.0844 - val_loss: 6.1132 - val_mae: 2.4723
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step - loss: 22.9575 - mae: 4.4218 - val_loss: 0.1035 - val_mae: 0.3218
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step - loss: 0.6096 - mae: 0.4650 - val_loss: 85.2959 - val_mae: 9.2353
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step - loss: 1.7001 - mae: 0.8408 - val_loss: 236.8977 - val_mae: 15.3914
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step - loss: 1.3800 - mae: 0.7009 - val_loss: 9.3854 - val_mae: 3.0636
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step - loss: 0.7375 - mae: 0.5886 - val_loss: 5.0498 - val_mae: 2.2470
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step - loss: 0.4213 - mae: 0.5474 - val_loss: 78.1327 - val_mae: 8.8392
Epoch 9/20
1/1 

In [34]:
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return np.sqrt(mse)

y_pred = resnet_model.predict(X_test)
rmse = calculate_rmse(y_test[:, 1], y_pred)
print(f"RMSE: {rmse}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
RMSE: 95.23023692061568


In [36]:
def plot_history(history, model_name):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

# Example
plot_history(history_cnn, "CNN")


NameError: name 'history_cnn' is not defined